In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
import cv2

In [3]:
from panelize import mark_components, panelize

In [4]:
BOARD_WIDTH = 85.6
BOARD_HEIGHT = 54
SPACING = 4.0
N_ROWS = 2
N_COLS = 2
EDGE_RAIL_HEIGHT = 5.0

INCH_TO_MM = 25.4

In [5]:
front_img = cv2.imread("paneltopgerbv fit.png").astype(np.float32)
panel_width = 175.2
pixel_width = panel_width / front_img.shape[1]

### Load PnP file

Exported using "centroids-screamingcircuits-smd" ULP in Fusion/Eagle.

In [6]:
pnp_fusion = pd.read_csv("pcb v55_centroid.csv", skiprows=7)
pnp_fusion = pnp_fusion.query("Layer == 'Top'")

pnp_fusion["LocationX"] = pnp_fusion["LocationX"] * INCH_TO_MM
pnp_fusion["LocationY"] = pnp_fusion["LocationY"] * INCH_TO_MM

pnp_fusion.rename(columns={"RefDes": "Designator", "LocationX": "Mid X", "LocationY": "Mid Y"}, inplace=True)

### Correct LED orientation (opposite of JLCPCB standard)

In [7]:
led_mask = pnp_fusion["Designator"].str.startswith("D")
flip_rotation = lambda r: (r + 180) % 360
pnp_fusion.loc[led_mask, "Rotation"] = pnp_fusion.loc[led_mask, "Rotation"].map(flip_rotation)

### Load BOM

In [8]:
bom = pd.read_csv("BOM_cnn_card_2020-11-28_19-21-05.csv", sep="\t")

### Panelize

In [9]:
pnp_panel, bom_panel = panelize(
    pnp_fusion,
    bom,
    board_width=BOARD_WIDTH,
    board_height=BOARD_HEIGHT,
    n_rows=N_ROWS,
    n_cols=N_COLS,
    spacing=SPACING,
    bottom_margin=EDGE_RAIL_HEIGHT + SPACING
)

### Export image

In [10]:
marked_img = mark_components(front_img, pnp_panel, pixel_width)

In [11]:
cv2.imwrite("panel_marked.png", marked_img)

True

### Save

In [12]:
bom_panel.to_csv("cnn_card_panel_2x2_bom_top.csv", index=None)

Add "mm" suffix to positions

In [13]:
pnp_panel["Mid X"] = pnp_panel["Mid X"].map(lambda x: str(x) + "mm")
pnp_panel["Mid Y"] = pnp_panel["Mid Y"].map(lambda x: str(x) + "mm")

In [14]:
pnp_panel.to_csv("cnn_card_panel_2x2_pnp_top.csv", index=None)